In [1]:
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim

import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

# find_lr function
import math
import matplotlib.pyplot as plt

# DDP
import torch.distributed as dist

### Automatic Mixed Precision (AMP)

In [2]:
from torch.cuda.amp import autocast, GradScaler

### Tensorboard

In [3]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter(log_dir="/data/runs/")

### Weights and Biases (wandb)

In [4]:
!pip install wandb
import wandb
wandb.login()

     |████████████████████████████████| 1.7 MB 6.3 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 13.4 MB/s eta 0:00:01
     |████████████████████████████████| 180 kB 104.6 MB/s eta 0:00:01
     |████████████████████████████████| 139 kB 109.4 MB/s eta 0:00:01
     |████████████████████████████████| 63 kB 3.2 MB/s s eta 0:00:01
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21502 sha256=03eb55b358405491e488426e7725f896f6aa6291e8018386fb32ffb354bb5a37
  Stored in directory: /root/.cache/pip/wheels/54/aa/01/724885182f93150035a2a91bce34a12877e8067a97baaf5dc8
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=6862d354320f93059f44bfe1b9457509f2784a6103596aab4a148bbaa13bffea
  Stored in directory: /root/.cache/pip/wheels/9f/69/d1/50b39b308a87998eaf5c1d9095e5a5bd2ad98501e2b7936d36
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4847 sha256=b4663b900e61cef8ce6855af8d7d6e4a3d1bfdf8

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [5]:
GPU=0

In [6]:
SEED=1

In [7]:
global_step=0

In [8]:
random.seed(SEED)
torch.manual_seed(SEED)
cudnn.deterministic = True

In [9]:
torch.cuda.device_count()

1

In [10]:
START_EPOCH = 0

In [11]:
!ls

checkpoint.pth.tar  model_best.pth.tar	train  val  wandb  worker.ipynb


In [12]:
ARCH = 'resnet18'
# ARCH = 'resnet152'
EPOCHS = 2 #200
LR = 0.1
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
PRINT_FREQ = 50
TRAIN_BATCH=500 #128
VAL_BATCH=500 #128
WORKERS=2
TRAINDIR="/data/hw9/train"
VALDIR="/data/hw9/val"

In [13]:
wandb.init(project='hw9_worker', 
           entity='rubyhan', 
           config = {
               "learning_rate": LR,
               "epochs": EPOCHS,
               "batch_size": TRAIN_BATCH,
               "momentum": MOMENTUM, 
               "weight_decay": WEIGHT_DECAY,
               "architecture": ARCH
           })

wandb: Currently logged in as: rubyhan (use `wandb login --relogin` to force relogin)

CondaEnvException: Unable to determine environment

Please re-run this command with one of the following options:

* Provide an environment name via --name or -n
* Re-run this command inside an activated conda environment.



In [14]:
# DDP
WORLD_SIZE = 2
BACKEND = 'nccl'
URL = 'tcp://172.31.26.78:1234'
RANK = 1

dist.init_process_group(backend = BACKEND, init_method= URL,
                                world_size= WORLD_SIZE, rank=RANK)

In [15]:
if not torch.cuda.is_available():
    print('GPU not detected.. did you pass through your GPU?')

In [16]:
torch.cuda.set_device(GPU)

In [17]:
cudnn.benchmark = True

In [18]:
def train(train_loader, model, criterion, optimizer, epoch):
    global global_step
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))

    scaler = GradScaler()
    
    # switch to train mode
    model.train()

    end = time.time()
    
    
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        optimizer.zero_grad()
        
        if GPU is not None:
            images = images.cuda(GPU, non_blocking=True)
        if torch.cuda.is_available():
            target = target.cuda(GPU, non_blocking=True)

#         # compute output
#         output = model(images)
#         loss = criterion(output, target)
        
        # WITH AMP
        with autocast():
            output = model(images)
            loss = criterion(output, target)

        # measure accuracy and record loss
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))

#         # compute gradient and do SGD step
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
        
        # use the scaler
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
        writer.add_scalar("Loss/train", loss, global_step = global_step)
        writer.add_scalar("acc1/train", top1.avg, global_step = global_step)
        writer.add_scalar("acc5/train", top5.avg, global_step = global_step)
        global_step += 1
        
        wandb.log({"Loss/train": loss, "acc1/train": top1.avg, "acc5/train": top5.avg})
        
        if i % PRINT_FREQ == 0:
            progress.display(i)

In [19]:
def validate(val_loader, model, criterion):
    global global_step
    
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            if GPU is not None:
                images = images.cuda(GPU, non_blocking=True)
            if torch.cuda.is_available():
                target = target.cuda(GPU, non_blocking=True)

            # compute output
            output = model(images)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % PRINT_FREQ == 0:
                progress.display(i)

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))
    
    writer.add_scalar("Loss/val", losses.avg, global_step = global_step)
    writer.add_scalar("acc1/val", top1.avg, global_step = global_step)
    writer.add_scalar("acc5/val", top5.avg, global_step = global_step)    
    
    wandb.log({"Loss/val": losses.avg, 'acc1/val': top1.avg, 'acc5/val': top5.avg})
    
    return top1.avg

In [20]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

In [21]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

In [22]:
class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [23]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = LR * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [24]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [25]:
imagenet_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
imagenet_std_RGB = [0.229, 0.224, 0.225]
cinic_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
cinic_std_RGB = [0.24205776, 0.23828046, 0.25874835]
cifar_mean_RGB = [0.4914, 0.4822, 0.4465]
cifar_std_RGB = [0.2023, 0.1994, 0.2010]

In [26]:
normalize = transforms.Normalize(mean=imagenet_mean_RGB, 
                                 std=imagenet_std_RGB)

In [27]:
IMG_SIZE = 224

In [28]:
NUM_CLASSES = 1000

In [29]:
model = models.__dict__[ARCH]()

In [30]:
inf = model.fc.in_features

In [31]:
model.fc = nn.Linear(inf, NUM_CLASSES)

In [32]:
model.cuda(GPU)

# Set model on BOTH GPUs
model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[GPU])

In [33]:
criterion = nn.CrossEntropyLoss().cuda(GPU)

In [34]:
optimizer = torch.optim.SGD(model.parameters(), LR,
                                momentum=MOMENTUM,
                                weight_decay=WEIGHT_DECAY)

In [35]:
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

In [36]:
transform_train = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.CenterCrop(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB),
])

In [37]:
train_dataset = datasets.ImageFolder(
    TRAINDIR, transform=transform_train)

In [38]:
transform_val = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB),
])

In [39]:
val_dataset = datasets.ImageFolder(
    VALDIR, transform=transform_val)

In [40]:
# train_loader = torch.utils.data.DataLoader(
#         train_dataset, batch_size=TRAIN_BATCH, shuffle=True,
#         num_workers=WORKERS, pin_memory=True, sampler=None)

# adding DDSampler to loader
train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=TRAIN_BATCH, shuffle=False,
        num_workers=WORKERS, pin_memory=True, sampler=torch.utils.data.distributed.DistributedSampler(train_dataset))

In [41]:
# val_loader = torch.utils.data.DataLoader(
#         val_dataset, batch_size=VAL_BATCH, shuffle=False,
#         num_workers=WORKERS, pin_memory=True, sampler=None)

# adding DDSampler to loader
val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=VAL_BATCH, shuffle=False,
        num_workers=WORKERS, pin_memory=True, sampler=torch.utils.data.distributed.DistributedSampler(val_dataset))

In [42]:
best_acc1 = 0

In [43]:
scaler = GradScaler()

def find_lr(init_value = 1e-8, final_value=10., beta = 0.98):
    num = len(train_loader)-1
    mult = (final_value / init_value) ** (1/num)
    lr = init_value
    optimizer.param_groups[0]['lr'] = lr
    avg_loss = 0.
    best_loss = 0.
    batch_num = 0
    losses = []
    log_lrs = []
    
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    meter_losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, meter_losses, top1, top5])
    
    # switch to train mode
    model.train()

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        batch_num += 1
        # measure data loading time
        data_time.update(time.time() - end)

        if GPU is not None:
            images = images.cuda(GPU, non_blocking=True)
        if torch.cuda.is_available():
            target = target.cuda(GPU, non_blocking=True)
        
        with autocast():
            # compute output
            output = model(images)
            loss = criterion(output, target)

        # measure accuracy and record loss
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        meter_losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))
        
        #Compute the smoothed loss
        avg_loss = beta * avg_loss + (1-beta) * loss.item()
        smoothed_loss = avg_loss / (1 - beta**batch_num)
        #Stop if the loss is exploding
        if batch_num > 1 and smoothed_loss > 4 * best_loss:
            return log_lrs, losses
        #Record the best loss
        if smoothed_loss < best_loss or batch_num==1:
            best_loss = smoothed_loss
            
        #Store the values
        losses.append(smoothed_loss)
        log_lrs.append(math.log10(lr))
        
        # compute gradient and do SGD step
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
        #Update the lr for the next step
        lr *= mult
        optimizer.param_groups[0]['lr'] = lr
        
        if i % PRINT_FREQ == 0:
            progress.display(i)
        
    return log_lrs, losses

# logs,losses = find_lr()
# plt.plot(logs[10:-5],losses[10:-5])

In [44]:
%%time

# start = time.time()
for epoch in range(START_EPOCH, EPOCHS):
#     adjust_learning_rate(optimizer, epoch)

    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch)

    # evaluate on validation set
    acc1 = validate(val_loader, model, criterion)

    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1)


    save_checkpoint({
        'epoch': epoch + 1,
        'arch': ARCH,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer' : optimizer.state_dict(),
    }, is_best)
    
    scheduler.step()
    print('lr: ' + str(scheduler.get_last_lr()))
    
    writer.add_scalar("lr", scheduler.get_last_lr()[0], global_step = global_step)
    
    wandb.log({'lr': scheduler.get_last_lr()[0]})
    
# print(f'{time.time() - start:.2f}s')

Epoch: [0][   0/1282]	Time 19.578 (19.578)	Data  4.175 ( 4.175)	Loss 7.0188e+00 (7.0188e+00)	Acc@1   0.20 (  0.20)	Acc@5   0.80 (  0.80)
Epoch: [0][  50/1282]	Time  0.665 ( 1.928)	Data  0.002 ( 0.945)	Loss 6.6397e+00 (6.8345e+00)	Acc@1   0.40 (  0.49)	Acc@5   2.40 (  1.81)
Epoch: [0][ 100/1282]	Time  1.412 ( 1.812)	Data  0.751 ( 0.980)	Loss 6.1260e+00 (6.6114e+00)	Acc@1   2.20 (  0.87)	Acc@5   7.60 (  3.22)
Epoch: [0][ 150/1282]	Time  0.756 ( 1.771)	Data  0.091 ( 0.993)	Loss 5.8674e+00 (6.4193e+00)	Acc@1   4.60 (  1.33)	Acc@5  11.00 (  4.77)
Epoch: [0][ 200/1282]	Time  0.715 ( 1.750)	Data  0.002 ( 0.990)	Loss 5.7007e+00 (6.2557e+00)	Acc@1   4.80 (  1.87)	Acc@5  12.20 (  6.32)
Epoch: [0][ 250/1282]	Time  0.691 ( 1.735)	Data  0.003 ( 0.985)	Loss 5.5211e+00 (6.1116e+00)	Acc@1   4.60 (  2.47)	Acc@5  14.20 (  7.93)
Epoch: [0][ 300/1282]	Time  0.728 ( 1.727)	Data  0.003 ( 0.984)	Loss 5.1714e+00 (5.9827e+00)	Acc@1   9.00 (  3.09)	Acc@5  21.40 (  9.54)
Epoch: [0][ 350/1282]	Time  0.701 ( 1.722

In [45]:
writer.close()
%reload_ext tensorboard
%tensorboard --logdir=/data/runs